# Move CSV Files From Datalake (Post Kafka Consumer) to SQL Database

In [ ]:
import numpy as np
import config               #hidden config file with keys and login credentials

**Extraction:** Read all csv files from Kafka producer/consumer into a Spark Dataframe.

In [ ]:
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/mnt/eddydoering/diabeetus/consumer_final/files/*.csv") 

**Transformation:** Replace strings "?" with NaN, so they'll be correctly imported as *Null* in SQL Database

In [ ]:
df = df.toPandas()                                                   #Pandas Dataframe necessary for Type change from Str to NaN
df.replace(to_replace='?', value = np.nan, inplace=True)
sparkDF = spark.createDataFrame(df)

**Load**: Load dataframe into SQL Database with JDBC API

In [ ]:
database = "group5"
table = "dbo.diabetestemp"
server = config.sql_server
jdbcUrl = f"jdbc:sqlserver://{server}:1433;databaseName={database};"
user = config.sql_login
password = config.sql_password

sparkDF.write.format("jdbc") \
    .mode("append") \
    .option("url", jdbcUrl) \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

Repeat for supplementary tables (Including Census Data)

In [ ]:
diag_df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/mnt/eddydoering/diabeetus/diag_diabetic.csv") 

table = "dbo.diag"

diag_df.write.format("jdbc") \
    .mode("append") \
    .option("url", jdbcUrl) \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
race_df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/mnt/eddydoering/diabeetus/2008_race_uninsured_census.csv") 

table = "dbo.race_uninsured_2008"

race_df.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbcUrl) \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
insurance_df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("/mnt/eddydoering/diabeetus/insurance_aggregate_2008.csv") 

table = "dbo.insurance_agg_2008"

insurance_df.write.format("jdbc") \
    .mode("overwrite") \
    .option("url", jdbcUrl) \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()